In [17]:
pip install langchain

Note: you may need to restart the kernel to use updated packages.


In [19]:
pip install langchain_groq


Note: you may need to restart the kernel to use updated packages.


In [1]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama3-70b-8192"
)

response = llm.invoke("The first person to land on moon was ...")
print(response.content)

That's an easy one!

The first person to set foot on the Moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the Moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to walk on the Moon.


In [23]:
pip install chromadb

Note: you may need to restart the kernel to use updated packages.


In [3]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://careers.nike.com/software-engineer-iii-unreal-engine/job/R-60944")
page_data = loader.load().pop().page_content

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills`, `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)
type(res.content)

[
    {
        "role": "Software Engineer III - Unreal Engine",
        "experience": "5+ years",
        "skills": ["C++ programming", "Unreal Engine framework", "web APIs", "asynchronous code", "3D graphics and rendering techniques", "software development best practices", "version control systems"],
        "description": "We are seeking multiple highly skilled Senior C++ Engineers with extensive experience in Unreal Engine to join our dynamic team. The ideal candidate will have a strong background in working with web APIs and asynchronous code. Experience in configurators, product visualization, video games, or VFX is highly desirable. This engineer will play a crucial role in refactoring our configurator and transitioning key classes into C++ to improve code reviews and overall efficiency."
    }
]


str

In [7]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

[{'role': 'Software Engineer III - Unreal Engine',
  'experience': '5+ years',
  'skills': ['C++ programming',
   'Unreal Engine framework',
   'web APIs',
   'asynchronous code',
   '3D graphics and rendering techniques',
   'software development best practices',
   'version control systems'],
  'description': 'We are seeking multiple highly skilled Senior C++ Engineers with extensive experience in Unreal Engine to join our dynamic team. The ideal candidate will have a strong background in working with web APIs and asynchronous code. Experience in configurators, product visualization, video games, or VFX is highly desirable. This engineer will play a crucial role in refactoring our configurator and transitioning key classes into C++ to improve code reviews and overall efficiency.'}]

In [9]:
wrapped_res = {"results": json_res}
print(type(wrapped_res))  # <class 'dict'>
print(wrapped_res['results'])  # your original list


<class 'dict'>
[{'role': 'Software Engineer III - Unreal Engine', 'experience': '5+ years', 'skills': ['C++ programming', 'Unreal Engine framework', 'web APIs', 'asynchronous code', '3D graphics and rendering techniques', 'software development best practices', 'version control systems'], 'description': 'We are seeking multiple highly skilled Senior C++ Engineers with extensive experience in Unreal Engine to join our dynamic team. The ideal candidate will have a strong background in working with web APIs and asynchronous code. Experience in configurators, product visualization, video games, or VFX is highly desirable. This engineer will play a crucial role in refactoring our configurator and transitioning key classes into C++ to improve code reviews and overall efficiency.'}]


In [11]:
type(wrapped_res)

dict

In [15]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [17]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

/Users/harichandanag/anaconda3/lib/python3.11/site-packages/opentelemetry/proto/collector/trace/v1/trace_service_pb2_grpc.py:26: RuntimeWarning: The grpc package installed is at version 1.48.2, but the generated code in opentelemetry/proto/collector/trace/v1/trace_service_pb2_grpc.py depends on grpcio>=1.63.2. Please upgrade your grpc module to grpcio>=1.63.2 or downgrade your generated code using grpcio-tools<=1.48.2. This warning will become an error in 1.65.0, scheduled for release on June 25, 2024.
  warnings.warn(


In [19]:
job = wrapped_res
print(job)

{'results': [{'role': 'Software Engineer III - Unreal Engine', 'experience': '5+ years', 'skills': ['C++ programming', 'Unreal Engine framework', 'web APIs', 'asynchronous code', '3D graphics and rendering techniques', 'software development best practices', 'version control systems'], 'description': 'We are seeking multiple highly skilled Senior C++ Engineers with extensive experience in Unreal Engine to join our dynamic team. The ideal candidate will have a strong background in working with web APIs and asynchronous code. Experience in configurators, product visualization, video games, or VFX is highly desirable. This engineer will play a crucial role in refactoring our configurator and transitioning key classes into C++ to improve code reviews and overall efficiency.'}]}


In [21]:
links = collection.query(query_texts=job['results'][0]['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/react-portfolio'}],
 [{'links': 'https://example.com/wordpress-portfolio'},
  {'links': 'https://example.com/full-stack-js-portfolio'}],
 [{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/full-stack-js-portfolio'}],
 [{'links': 'https://example.com/wordpress-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}]]

In [22]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Harichandana a business development executive at We3. We3 is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.Introduce yourself also.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Harichandana, BDE at We3. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Here is the cold email:

Subject: Expert Unreal Engine Developers for Your Configurator Project

Dear Hiring Manager,

I'm Harichandana, a Business Development Executive at We3, an AI & Software Consulting company dedicated to facilitating the seamless integration of business processes through automated tools. We've empowered numerous enterprises with tailored solutions, fostering scalability, process optimization, cost reduction, and heightened overall efficiency.

I came across your job posting for a Software Engineer III - Unreal Engine, and I'm excited to introduce our team at We3 as a potential partner to fulfill your requirements. Our experts possess the necessary skills and experience to refactor your configurator and transition key classes into C++ to improve code reviews and overall efficiency.

Our team has extensive experience in C++ programming, Unreal Engine framework, web APIs, asynchronous code, 3D graphics and rendering techniques, software development best practices, a